In [ ]:
import numpy as np
import scipy as sp
from scipy.ndimage import gaussian_filter
from scipy.signal import butter, filtfilt, find_peaks, freqz, sosfilt, bessel
import matplotlib.pyplot as plt
from cupyx.scipy.ndimage import gaussian_filter as gaussian_filter_gpu
import cupy as cp
%matplotlib widget

nPoints = 2**16

noise = np.random.normal(0, 1, nPoints)
#spur
noise[3*nPoints//4] = 1000
undulations = 6*np.sin(4*np.pi*np.arange(0,nPoints,1)/(nPoints)) + np.linspace(1,20,nPoints)
wideSig = 6*np.exp(-(np.linspace(-nPoints//2,nPoints//2,nPoints)**2)/(2*2)**2)

signal = undulations + noise + 1 + wideSig




plt.figure()
plt.plot(signal)


In [ ]:
fc_numBins = 30
fcNorm = 2./(fc_numBins)
'''
b, a = bessel(6, fcNorm, 'highpass', analog=False, norm='phase', output = 'ba')
besselHpf = filtfilt(b, a, signal)
'''
b, a = butter(6, fcNorm, 'highpass', analog = False)
# Apply the Butterworth filter to the spectrum
butterHpf = filtfilt(b, a, signal)

signal_gpu = cp.array(signal)
gausLpf_gpu = gaussian_filter_gpu(signal_gpu, sigma = 30)
gausLpf = gausLpf_gpu.get()
gausHpf = signal - gausLpf


print('butter MAD = ',sp.stats.median_abs_deviation(butterHpf))
print('gauss MAD = ', sp.stats.median_abs_deviation(gausHpf))
plt.figure()
plt.plot(signal, label = 'signal')
#plt.plot(besselHpf, label = 'bessel')
plt.plot(gausHpf, label = 'gauss')
plt.plot(butterHpf, label = 'butter', alpha = .5)
plt.legend(loc = 'upper right')


In [ ]:
%%timeit
signal_gpu = cp.array(signal)
gausLpf_gpu = gaussian_filter_gpu(signal_gpu, sigma = 300)
gausLpf = gausLpf_gpu.get()
gausHpf = signal - gausLpf

In [ ]:
%%timeit
gausLpf = gaussian_filter(signal, sigma = 300)
gausHpf = signal - gausLpf

In [ ]:
#gaussian filter is a low pass
gausFilt = gaussian_filter(signal, sigma=300)
gausHpf = signal - gausFilt

plt.figure()
plt.plot(signal, label = 'sig')
plt.plot(gausHpf, label = 'gauss hpf')

plt.figure()
plt.plot(noise)
plt.plot(gausHpf, alpha = .5)

In [ ]:
plt.figure()
plt.plot(butterFilt, label = 'butter')
plt.plot(gausHpf, alpha = .5, label = 'gauss')
plt.legend()

plt.figure()
plt.plot(signal - butterFilt, label = 'butter lpf')
plt.plot(gausFilt, alpha = .5, label = 'gauss lpf')
plt.legend()